# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Digital Methods	Lecture	lecture-dm	National University of Athens	3/12/22	Athens, Greece		Gave a lecture to MSc Psychology Students of the NUA on Digital Methods for the Social Sciences
(AoIR2022) The Role of Citizens in Platform Governance: A Case Study of Public Consultations on European Online Content Regulation	Presentation	aoir2022-pre1	Association of Internet Researchers	1/11/22	Dublin, Ireland		Participated in the AoIR2022 workshop on Radical Approaches to Platform Governance
(ICA2022) How Google and Facebook Capture Journalism	Presentation	ica2022-pre2	International Communication Association	28/5/22	Paris, France		Participated in the ICA 2022 Conference in Paris (Panel on Media Capture)
(ICA2022) Funding Intermediaries: How Google and Facebook Capture Journalism	Presentation	ica2022-pre3	International Communication Association	26/5/22	Paris, France		Participated in the ICA 2022 Conference in Paris (Preconference: From 

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Digital Methods,Lecture,lecture-dm,National University of Athens,3/12/22,"Athens, Greece",NaN,Gave a lecture to MSc Psychology Students of t...
1,(AoIR2022) The Role of Citizens in Platform Go...,Presentation,aoir2022-pre1,Association of Internet Researchers,1/11/22,"Dublin, Ireland",NaN,Participated in the AoIR2022 workshop on Radic...
2,(ICA2022) How Google and Facebook Capture Jour...,Presentation,ica2022-pre2,International Communication Association,28/5/22,"Paris, France",NaN,Participated in the ICA 2022 Conference in Par...
3,(ICA2022) Funding Intermediaries: How Google a...,Presentation,ica2022-pre3,International Communication Association,26/5/22,"Paris, France",NaN,Participated in the ICA 2022 Conference in Par...
4,(ICA2022) The Role of Citizens in Platform Gov...,Presentation,ica2022-pre1,International Communication Association,25/5/22,"Paris, France",NaN,Participated in the ICA 2022 Conference in Par...
5,JOLT Amsterdam Conference,Presentation,jolt-amsterdam,JOLT,1/4/22,"Amsterdam, Netherlands",NaN,Participated in the JOLT April 2022 Conference...
6,Social media as a mirror for the 21st century ...,Talk,1821-observatory,Online,16/11/21,"Athens, Greece",NaN,NaN
7,Content moderation and censorship on social me...,Article,como-greece-ena,Online,16/11/21,"Athens, Greece",https://www.enainstitute.org/__________-¹_____...,Article for a Greek news outlet
8,European Journal of Communication Symposium,Conference,ejc-symposium,European Journal of Communication,21/10/21,"Toulouse, France",NaN,Participated in a symposium of the EJC for an ...
9,Anti-vaxxers and sceptics on Greek-language so...,Article,rosa-antivax,Online,17/10/21,"Athens, Greece",https://www.rosa.gr/ereyna/antiemvoliastes-kai...,Article for a Greek news outlet


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

20-aoir2020.md              21-efsyn-art.md
20-fujo-art1.md             21-ejc-symposium.md
20-ina-art1.md              21-epohi-art.md
20-ina-art2.md              21-jolt-pamplona.md
20-jolt-london.md           21-jolt-toulouse.md
20-panel-jolt2020.md        21-panteion-lecture.md
20-tpp-art1.md              21-plat-gov-presentation.md
2012-03-01-talk-1.md        21-rosa-antivax.md
2013-03-01-tutorial-1.md    21-tpp-art2.md
2014-02-01-talk-2.md        22-aoir2022-pre1.md
2014-03-01-talk-3.md        22-ica2022-pre1.md
21-1821-observatory.md      22-ica2022-pre2.md
21-aoir2021-pr1.md          22-ica2022-pre3.md
21-como-greece-ena.md       22-jolt-amsterdam.md
21-covid-check-bai.md       22-lecture-dm.md
21-covid-check-fujo.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
